In [ ]:
import pickle
import numpy as np
import xarray as xr
import rioxarray
with open('../data/2_img_test.pkl', 'rb') as f:
    ds = pickle.load(f)

In [ ]:
ds = ds.set_xindex('relative_orbit')

In [ ]:
ds.sel(relative_orbit = 20)

In [ ]:
ds.where(ds.flight_dir == 'ascending', drop = True)